In [4]:
#!pip install pandas-datareader
#!pip install bokeh

## ВЫПОЛНИТЬ ЗАКОММЕНТИРОВАННЫЕ НИЖЕ КОМАНДЫ, ЕСЛИ ВИЗУАЛИЗАЦИЯ НЕ ОТОБРАЗИЛАСЬ ИЛИ НЕТ УСТАНОВЛЕННЫХ БИБЛИОТЕК
!jupyter labextension install @jupyter-widgets/jupyterlab-manager 
!jupyter labextension install bqplot

!pip install bqplot
!pip install ipywidgets

import numpy as np
import pandas as pd
import pandas_datareader as pdr
import pandas_datareader.data as web
from IPython.display import display
from datetime import datetime, timezone
from datetime import timedelta
from bqplot import (OHLC, DateScale, LinearScale, Axis,Toolbar, Figure, PanZoom, Lines, Scatter)
from ipywidgets import ToggleButtons


-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/

Building jupyterlab assets (production, minimized)


Building jupyterlab assets (production, minimized)


In [2]:

## функция для получения датафрейма 
def get_data(ticker, start = '2015-1-1', stop = '2019-12-31'):
    start = datetime.strptime(start,'%Y-%m-%d')
    stop = datetime.strptime(stop,'%Y-%m-%d')
    return(web.get_data_yahoo(ticker, start=start, end=stop))

## функция для автоматического скейлинга по обеим осям
def py_update(new):
        tb0._panzoom.scales={'x': [x_date], 'y': [y_linear]}

## бот
def trader(df,money = 100000,stop_loss = 7, short_window = 30, long_window = 90):
    # frame fixing
    df.columns = df.columns.map(lambda x: x.capitalize())
    df.index = df.index.map(lambda x: datetime.strftime(x, '%Y-%m-%d'))
    df.index = df.index.map(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    # signal determination with stop-loss implemented
    short_crv = df.Close.rolling(short_window).mean()
    long_crv = df.Close.rolling(long_window).mean()
    asign = np.sign(short_crv - long_crv).dropna()
    signchange = (~(np.roll(asign, 1) - asign).isin([0,np.nan])).values
    asing = asign[signchange]
    dates = asing.index
    df = df.reset_index()


    df_test = pd.DataFrame(columns = ['date', 'signal', 'num_shares','share_price',
                                      'share_value', 'rest', 'cash'])

    cap = money

    stop_loss /= 100 

    if(asign[signchange].iloc[0] < 0):
        price_sale = np.round(df.Open[df.Date.shift(1) == dates[0]],2).values[0]
        df_test = pd.concat([df_test, pd.DataFrame({'date': [dates[0],
                                                             df.Date[df.Date.shift(1) 
                                                                     == dates[0]].values[0]],
                                                    'signal':['sig sale', 'sale'],
                                                    'num_shares':[0,0],
                                                    'share_price':[np.nan, price_sale],
                                                    'share_value':[np.nan, 0],
                                                    'rest':[0, 0],
                                                    'cash':[cap, cap]})])


    for i in dates:
        sign = asign[asign.index == i].values
        if sign == 1. and dates.get_loc(i) != (len(dates) - 1):
            price_buy = np.round(df.Open[df.Date.shift(1) == dates[dates.get_loc(i)]],2).values[0]
            price_sale = np.round(df.Open[df.Date.shift(1) == dates[dates.get_loc(i) + 1]],2).values[0]
            amount_action = int(cap // price_buy)
            final_cash = cap
            for j in pd.bdate_range(df.Date[df.Date.shift(1) == dates[dates.get_loc(i)]].values[0],
                                    df.Date[df.Date.shift(1) == dates[dates.get_loc(i) + 1]].values[0]):
                try: 
                    if (df.Low[df.Date == j]/price_buy < (1 - stop_loss)).values[0]:
                        final_cash = (cap - price_buy * amount_action 
                                      + np.round((1 - stop_loss) * price_buy,2)
                                      * amount_action)
                        df_test = pd.concat([df_test, pd.DataFrame({'date': [i,df.Date[df.Date.shift(1)
                                                                                       == dates[dates.get_loc(i)]]
                                                                             .values[0],
                                                                             j,dates[dates.get_loc(i) + 1],
                                                                             df.Date[df.Date.shift(1) 
                                                                                     == dates[dates.get_loc(i) + 1]]
                                                                             .values[0]],
                                                                    'signal':['sig buy', 'buy', 'stop-loss',
                                                                              'sig sale', 'sale'],
                                                                    'num_shares':[0,amount_action,
                                                                                  amount_action,0,0],
                                                                    'share_price':[np.nan, price_buy,
                                                                                   np.round((1 - stop_loss)
                                                                                            *price_buy,2),
                                                                                   np.nan, price_sale],
                                                                    'share_value':[np.nan, price_buy * amount_action,
                                                                                   np.round((1 - stop_loss)
                                                                                            * price_buy,2) 
                                                                                   * amount_action, np.nan, 0],
                                                                    'rest':[0, cap - price_buy * amount_action, 
                                                                            cap - price_buy * amount_action, 0, 0],
                                                                    'cash':[cap, cap - price_buy * amount_action,
                                                                            final_cash,final_cash,final_cash]})])
                        break
                except IndexError:
                    continue
            try:
                if cap == final_cash :
                    final_cash = (cap - price_buy * amount_action 
                                  + price_sale * amount_action)
                    df_test = pd.concat([df_test, pd.DataFrame({'date': [i,df.Date[df.Date.shift(1)
                                                                                   == dates[dates.get_loc(i)]]
                                                                         .values[0],
                                                                         dates[dates.get_loc(i) + 1],
                                                                         df.Date[df.Date.shift(1) 
                                                                                 == dates[dates.get_loc(i) + 1]]
                                                                         .values[0]],
                                                                'signal':['sig buy', 'buy',
                                                                          'sig sale', 'sale'],
                                                                'num_shares':[0,amount_action,
                                                                              amount_action,
                                                                              amount_action],
                                                                'share_price':[np.nan, price_buy,
                                                                               np.nan, price_sale],
                                                                'share_value':[np.nan, price_buy * amount_action,
                                                                               np.nan, price_sale * amount_action],
                                                                'rest':[0, cap - price_buy * amount_action, 
                                                                        cap - price_buy * amount_action,
                                                                        cap - price_buy * amount_action],
                                                                'cash':[cap, cap - price_buy * amount_action,
                                                                        cap - price_buy * amount_action,
                                                                        final_cash]})])
            except IndexError:
                continue
            cap = df_test.cash.iloc[-1]

    df_test.index = range(len(df_test)) 
    df_test = df_test[:df_test[df_test.signal.isin(['sale','stop-loss']) 
                               & df_test.num_shares > 0].index[-1] + 1]
    
    print('Итоговая стоимость после последней продажи: ' + str(np.round(df_test.cash.iloc[-1],2)))
    print('------------------------------------------------------------------')
    print('Итоговый процент прироста вложений: ' + str(np.round((df_test.cash.iloc[-1] 
                                                                 / df_test.cash.iloc[0] - 1)
                                                                * 100, 2)) + ' %')
    
    
    display(df_test)
    
    ## visualization:
    x_date = DateScale()
    y_linear = LinearScale()
    y_linear_act = LinearScale()

    line_short = Lines(x = short_crv.index, y = short_crv.values,
                       colors = ['orange'], scales={'x':x_date, 'y':y_linear})
    line_long = Lines(x = long_crv.index, y = long_crv.values,
                      colors = ['brown'], scales={'x':x_date, 'y':y_linear})

    activ_x = [df.Date.iloc[0],df_test[df_test.signal == 'buy'].date.iloc[0]]
    activ_y = np.array([0,0])
    for i in df_test[df_test.signal.isin(['buy', 'sale', 'stop-loss'])].date.values:
        activ_x.extend([i, i])
        if activ_y[len(activ_y) - 1] == 0:
            activ_y = np.append(activ_y, [np.where(df_test.signal[df_test.date == i] == 'buy', 0, 0),
                                          np.where(df_test.signal[df_test.date == i] == 'buy', 1, 0)])
        else: 
            activ_y = np.append(activ_y, [np.where(df_test.signal[df_test.date == i] == 'buy', 0, 1),
                                          np.where(df_test.signal[df_test.date == i] == 'buy', 1, 0)])
    activ_x.extend([df.Date.iloc[-1], df.Date.iloc[-1]])
    activ_y = np.append(activ_y, [0, 0])

    activity = Lines(x = activ_x, y = activ_y,
                     colors = ['black'], scales={'x':x_date, 'y':y_linear_act})

    sigs_buy = Scatter(x = df_test.date[df_test.signal.isin(['sig buy'])],
                       y = df.High[df.Date.isin(df_test.date[df_test.signal.isin(['sig buy'])].values)],
                       colors = ['green'], marker = 'triangle-up',
                       scales={'x':x_date, 'y':y_linear})

    sigs_sale = Scatter(x = df_test.date[df_test.signal.isin(['sig sale'])],
                        y = df.High[df.Date.isin(df_test.date[df_test.signal.isin(['sig sale'])].values)],
                        colors = ['red'], marker = 'triangle-down',
                        scales={'x':x_date, 'y':y_linear})
    
    ohlc = OHLC(x=df.Date.values, y=df[["Open","High","Low","Close"]],
                scales={'x':x_date, 'y':y_linear},
                marker="candle",
                stroke="dodgerblue",
                stroke_width=1.5,
                colors=["lime", "tomato"],
                )

    ax_x = Axis(scale=x_date, label="Date",
                label_offset="35px", grid_color="gray",
                )
    ax_y = Axis(scale=y_linear, label="Price",
                orientation="vertical", label_offset="35px",
                grid_color="gray",
                tick_format="0.1f")
    
    try:
        sigs_stop = Scatter(x = df_test.date[df_test.signal.isin(['stop-loss'])],
                            y = df.High[df.Date.isin(df_test.date[df_test.signal.isin(['stop-loss'])].values)],
                            colors = ['black'], marker = 'triangle-down',
                            scales={'x':x_date, 'y':y_linear})
        fig = [Figure(marks=[ohlc, line_short, line_long, sigs_buy, sigs_sale, sigs_stop],
                  axes=[ax_x, ax_y], title="Trading protocol Chart",
                  fig_margin= dict(top=60, bottom=40, left=50, right=20)
                 ), 
           Figure(marks=[activity],
                  axes=[ax_x], title="Activity protocol Chart",
                  fig_margin= dict(top=60, bottom=40, left=50, right=20)
                 )]
    except ValueError:
        fig = [Figure(marks=[ohlc, line_short, line_long, sigs_buy, sigs_sale],
                  axes=[ax_x, ax_y], title="Trading protocol Chart",
                  fig_margin= dict(top=60, bottom=40, left=50, right=20)
                 ), 
           Figure(marks=[activity],
                  axes=[ax_x], title="Activity protocol Chart",
                  fig_margin= dict(top=60, bottom=40, left=50, right=20)
                 )]

    

    tb = ToggleButtons()
    tb.observe(py_update, "value")

    

    fig[0].layout.height = '600px'
    fig[1].layout.height = '200px'

    tb0 = Toolbar(figure=fig[0])
    tb1 = Toolbar(figure=fig[1])

    display(fig[0], tb0, tb)
    display(fig[1])
    return(df_test)

In [5]:
#Result output:
df = get_data('AAPL')

vis = trader(df, stop_loss = 20 )

Итоговая стоимость после последней продажи: 110437.27
------------------------------------------------------------------
Итоговый процент прироста вложений: 10.44 %


,date,signal,num_shares,share_price,share_value,rest,cash
0,2015-07-10 00:00:00,sig sale,0,NaN,NaN,0,100000
1,2015-07-13 00:00:00,sale,0,31.26,0.0,0,100000
2,2015-11-18 00:00:00,sig buy,0,NaN,NaN,0.0,100000.0
3,2015-11-19 00:00:00,buy,3400,29.41,99994.0,6.0,6.0
4,2015-12-31 00:00:00,sig sale,3400,NaN,NaN,6.0,6.0
5,2016-01-04 00:00:00,sale,3400,25.65,87210.0,6.0,87216.0
6,2016-04-06 00:00:00,sig buy,0,NaN,NaN,0.0,87216.0
7,2016-04-07 00:00:00,buy,3172,27.49,87198.28,17.72,17.72
8,2016-05-19 00:00:00,sig sale,3172,NaN,NaN,17.72,17.72
9,2016-05-20 00:00:00,sale,3172,23.66,75049.52,17.72,75067.24


Figure(axes=[Axis(grid_color='gray', label='Date', label_offset='35px', scale=DateScale()), Axis(grid_color='g…

Toolbar(figure=Figure(axes=[Axis(grid_color='gray', label='Date', label_offset='35px', scale=DateScale()), Axi…

ToggleButtons(options=(), value=None)

Figure(axes=[Axis(grid_color='gray', label='Date', label_offset='35px', scale=DateScale())], fig_margin={'top'…